In [1]:
# -*- coding: utf8 -*-

In [2]:
import numpy as np
from gensim.models import Word2Vec
from medicalDataLoad import *

### 1. load入院数据

In [3]:
import pandas as pd
#829177
diaPath = "hospitalData/df_hospital_20171219_diseasematchKeywordNum.csv"
pd_dia = pd.read_csv(diaPath,  encoding='utf8').drop_duplicates()
pd_dia.shape

(56416, 33)

In [4]:
def loadCodeType(fileName):
    fileList = list(open(fileName,"r").readlines())
    #print(fileList)
    firstCode2IndexDict = dict()
    firstIndex2TypeDict = dict()
    for i in range(len(fileList)):
        itemArr = fileList[i].strip().split(" ")
        firstCode2IndexDict[itemArr[0]] = itemArr[1]
        firstIndex2TypeDict[itemArr[1]] = itemArr[2]
    return firstCode2IndexDict, firstIndex2TypeDict
fileName = "datasets/firstCode2Index2Type.txt"
firstCode2IndexDict, firstIndex2TypeDict = loadCodeType(fileName)
#print(firstCode2IndexDict)
print(firstIndex2TypeDict)

{'4': '精神和行为障碍', '0': '某些传染病和寄生虫病', '2': '血液及造血器官疾病和某些涉及免疫机制的疾患', '13': '泌尿生殖系统疾病', '7': '耳和乳突疾病', '3': '内分泌、营养和代谢疾病', '15': '起源于围生期的某些情况', '11': '皮肤和皮下组织病', '14': '妊娠分娩及产褥期', '9': '呼吸系统疾病', '20': '影响健康状态和保健机构接触的因素', '6': '眼和附器疾病', '5': '神经系统疾病', '1': '肿瘤', '12': '肌肉和骨骼系统和结缔组织疾病', '10': '消化系统疾病', '17': '症状、体症和临床与实验室异常所见，不可归类在他处者', '18': '损伤、中毒和外因的某些其它后果', '19': '疾病和死亡的外因', '8': '循环系统疾病', '16': '先天畸形、变性和染色体异常'}


## 2.选取使用的字段

In [5]:
pd_dia_dis = pd_dia[["FirstCode","disease","chief_complaintKey","present_illnessKey","admission_situationKey","treat_processKey",
                     "discharge_situationKey","discharge_orderKey"]]
pd_dia_dis.head(3)

,FirstCode,disease,chief_complaintKey,present_illnessKey,admission_situationKey,treat_processKey,discharge_situationKey,discharge_orderKey
0,I00-I99,脑梗死,NaN,NaN,侧肢体无力 双侧 病理征 患者病情 伸舌居中 肌力肌张力 心慌胸闷 无恶心呕吐 查房 肢体抽...,侧肢体无力 双侧 病理征 患者病情 伸舌居中 肌力肌张力 心慌胸闷 无恶心呕吐 查房 肢体抽...,侧肢体无力 双侧 病理征 患者病情 伸舌居中 肌力肌张力 心慌胸闷 无恶心呕吐 查房 肢体抽...,侧肢体无力 双侧 病理征 患者病情 伸舌居中 肌力肌张力 心慌胸闷 无恶心呕吐 查房 肢体抽...
1,I00-I99,脑梗死,NaN,NaN,左侧 未见异常 切题 腱反射 病理反射阴性 右侧顶叶 CT 记忆力下降 伸舌 轮替动作 肢体...,脑梗死,入院 脑萎缩 血管动脉硬化 双侧脑室 空蝶鞍 化验结果 桥脑 大脑后动脉狭窄 放射冠区 对症...,脑梗死 脑萎缩 双膝 3. 脑动脉硬化 退行性骨关节炎 狭窄
2,I00-I99,脑梗死,NaN,NaN,双侧 查房 双侧瞳孔 浅感觉 无恶心呕吐 心律齐 伸舌居中 双肺呼吸音清 巴氏征阴性 等大 ...,双侧 查房 双侧瞳孔 浅感觉 无恶心呕吐 心律齐 伸舌居中 双肺呼吸音清 巴氏征阴性 等大 ...,双侧 查房 双侧瞳孔 浅感觉 无恶心呕吐 心律齐 伸舌居中 双肺呼吸音清 巴氏征阴性 等大 ...,双侧 查房 双侧瞳孔 浅感觉 无恶心呕吐 心律齐 伸舌居中 双肺呼吸音清 巴氏征阴性 等大 ...


#### list按比例随机切分 

In [24]:
#随机按照指定的比例切分训练集合和测试集合
def doDatasetSplit(x,x_admin, y, test_size):
    X_num=len(x)
    train_index=[ i for i in range(X_num)]
    #print(train_index)
    test_index=[]
    test_num=int(X_num*test_size)
    #print("test=",test_num)
    #随机选取index
    for i in range(test_num):
        randomIndex=int(np.random.uniform(0,len(train_index)))
        if randomIndex not in train_index:
            continue
        test_index.append(train_index[randomIndex])
        #print(randomIndex)
        train_index.remove(randomIndex)
    #train,test的index是抽取的数据集X的序号
    x_disease_train = [x[item] for item in train_index]
    x_admin_train = [x_admin[item] for item in train_index]
    x_disease_test = [x[item] for item in test_index]
    x_admin_test = [x_admin[item] for item in test_index]
    
    y_train = [y[item] for item in train_index]
    y_test = [y[item] for item in test_index]

    return x_disease_train, x_admin_train, x_disease_test,x_admin_test, y_train, y_test

### 3.全量数据集合切分成训练集和测试集

In [25]:
#1.load all datasets  2.do split on each first type datasets 3.merge to a final train and test sets 4.word to vector 5.do rf train and test
#1.load all dataset pd_dia_dis
def classifyDatasetsTF(pd_dia_dis,columnName, rate):
    x_disease_trainAll = list()
    x_admin_trainAll = list()
    x_disease_testAll = list()
    x_admin_testAll = list()
    y_trainAll = list()
    y_testAll = list()
    
    for item in firstCode2IndexDict.keys():
        #2.do split oon each first type, disease and admission_situation
        #print("item=", item,firstCode2IndexDict[item])
        pd_tmp = pd_dia_dis[pd_dia_dis["FirstCode"]==item]
        x_disease = pd_tmp['disease'].tolist()
        #print("dis num=",len(x_disease))
        x_admin = pd_tmp[columnName].tolist()

        #x_admin = pd_tmp["admission_situationKey"].tolist()
        label = firstCode2IndexDict[item]
        nums = len(x_disease)
        if nums==0:
            continue
        y = [label]*nums
        x_disease_train, x_admin_train, x_disease_test,x_admin_test, y_train, y_test = doDatasetSplit(x_disease,x_admin, y, rate)
        #3.merge 
        x_disease_trainAll.extend(x_disease_train)
        x_admin_trainAll.extend(x_admin_train)
        x_disease_testAll.extend(x_disease_test)
        x_admin_testAll.extend(x_admin_test)
        y_trainAll.extend(y_train)
        y_testAll.extend(y_test)
    print("train size=", len(y_trainAll), "test size=", len(y_testAll))
    print("sum=",len(y_trainAll)+len(y_testAll))
        
    return x_disease_trainAll, x_admin_trainAll,x_disease_testAll,x_admin_testAll, y_trainAll, y_testAll
rate = 0.2
columnName = "admission_situationKey"
x_disease_trainAll, x_admin_trainAll,x_disease_testAll,x_admin_testAll, y_trainAll, y_testAll = classifyDatasetsTF(pd_dia_dis,columnName,rate)

train size= 46291 test size= 10125
sum= 56416


In [28]:
def doPrepareCNN(x_disease_trainAll, x_admin_trainAll, y_trainAll, filePath):
    f = open(filePath+".feature", "w")
    f_label =open(filePath+".label" ,"w")
    for i in range (len(x_disease_trainAll)):
        dis = x_disease_trainAll[i]
        adm = x_admin_trainAll[i]
       
        y_train = y_trainAll[i]
        
        y_trainStr = ("%s " %(y_train))
        #featureStr = ("%s%s" %(str(dis)+" " + str(adm) + " __label__", y_trainStr))
                               
        featureStr = ("%s" %(str(dis)+" " + str(adm)))
        labelStr = ("%s" %(y_trainStr))
        f.write(str(featureStr)+"\n")
        f_label.write(labelStr+"\n")
    f.flush()
    f.close()

filePath = "cnnModel/datasets/train1"
doPrepareCNN(x_disease_trainAll, x_admin_trainAll, y_trainAll, filePath )
filePathtest = "cnnModel/datasets/test1"
doPrepareCNN(x_disease_testAll,x_admin_testAll,y_testAll, filePathtest )

In [14]:
rate = 0.2
columnName = "discharge_situationKey"
x_disease_trainAll, x_dissituation_trainAll,x_disease_testAll,x_dissituation_testAll, y_trainAll, y_testAll = classifyDatasetsTF(pd_dia_dis,columnName,rate)

train size= 46282 test size= 10134
sum= 56416


In [18]:
rate = 0.2
columnName = "treat_processKey"
x_disease_trainAll, x_treat_trainAll,x_disease_testAll,x_treat_testAll, y_trainAll, y_testAll = classifyDatasetsTF(pd_dia_dis,columnName,rate)
rate = 0.2
columnName = "discharge_orderKey"
x_disease_trainAll, x_disorder_trainAll,x_disease_testAll,x_disorder_testAll, y_trainAll, y_testAll = classifyDatasetsTF(pd_dia_dis,columnName,rate)

train size= 46240 test size= 10176
sum= 56416
train size= 46251 test size= 10165
sum= 56416


In [33]:
#随机按照指定的比例切分训练集合和测试集合
def doDatasetSplitAll(x,x_admin,x_dissituation,x_treat,x_disorder,y, test_size):
    X_num=len(x)
    train_index=[ i for i in range(X_num)]
    #print(train_index)
    test_index=[]
    test_num=int(X_num*test_size)
    #print("test=",test_num)
    #随机选取index
    for i in range(test_num):
        randomIndex=int(np.random.uniform(0,len(train_index)))
        if randomIndex not in train_index:
            continue
        test_index.append(train_index[randomIndex])
        #print(randomIndex)
        train_index.remove(randomIndex)
    #train,test的index是抽取的数据集X的序号
    x_disease_train = [x[item] for item in train_index]
    x_admin_train = [x_admin[item] for item in train_index]
    x_dissituation_train = [x_dissituation[item] for item in train_index]
    x_treat_train = [x_treat[item]  for item in train_index]
    x_disorder_train = [x_disorder[item] for item in train_index]
    
    x_disease_test = [x[item] for item in test_index]
    x_admin_test = [x_admin[item] for item in test_index]
    x_dissituation_test = [x_dissituation[item] for item in test_index]
    x_treat_test = [x_treat[item]  for item in test_index]
    x_disorder_test = [x_disorder[item] for item in test_index]
    
    y_train = [y[item] for item in train_index]
    y_test = [y[item] for item in test_index]

    return x_disease_train, x_admin_train,x_dissituation_train,x_treat_train,x_disorder_train, x_disease_test,x_admin_test,x_dissituation_test,x_treat_test,x_disorder_test, y_train, y_test

In [61]:
#1.load all datasets  2.do split on each first type datasets 3.merge to a final train and test sets 4.word to vector 5.do rf train and test
#1.load all dataset pd_dia_dis
def classifyDatasetsTFAll(pd_dia_dis,admin,dissituation,treat,disorder, rate):
    x_disease_trainAll = list()
    x_admin_trainAll = list()
    x_dissituation_trainAll = list()
    x_treat_trainAll = list()
    x_disorder_trainAll = list()
    
    x_disease_testAll = list()
    x_admin_testAll = list()
    x_dissituation_testAll = list()
    x_treat_testAll = list()
    x_disorder_testAll = list()
    
    y_trainAll = list()
    y_testAll = list()
    
    for item in firstCode2IndexDict.keys():
        #2.do split oon each first type, disease and admission_situation
        #print("item=", item,firstCode2IndexDict[item])
        pd_tmp = pd_dia_dis[pd_dia_dis["FirstCode"]==item]
        x_disease = pd_tmp['disease'].tolist()
        #print("dis num=",len(x_disease))
        x_admin = pd_tmp[admin].tolist()
        x_dissituation = pd_tmp[dissituation].tolist()
        x_treat = pd_tmp[treat].tolist()
        x_disorder = pd_tmp[disorder].tolist()

        #x_admin = pd_tmp["admission_situationKey"].tolist()
        label = firstCode2IndexDict[item]
        nums = len(x_disease)
        if nums==0:
            continue
        y = [label]*nums
        x_disease_train, x_admin_train,x_dissituation_train,x_treat_train,x_disorder_train, x_disease_test,\
            x_admin_test,x_dissituation_test,x_treat_test,x_disorder_test, y_train, y_test=doDatasetSplitAll(
            x_disease,x_admin, x_dissituation,x_treat,x_disorder,y, rate)
        #3.merge 
        x_disease_trainAll.extend(x_disease_train)
        x_admin_trainAll.extend(x_admin_train)
        x_dissituation_trainAll.extend(x_dissituation_train)
        x_treat_trainAll.extend(x_treat_train)
        x_disorder_trainAll.extend(x_disorder_train)
        
        x_disease_testAll.extend(x_disease_test)
        x_admin_testAll.extend(x_admin_test)
        x_dissituation_testAll.extend(x_dissituation_test)
        x_treat_testAll.extend(x_treat_test)
        x_disorder_testAll.extend(x_disorder_test)
        
        y_trainAll.extend(y_train)
        y_testAll.extend(y_test)
    print("train size=", len(y_trainAll), "test size=", len(y_testAll))
    print("sum=",len(y_trainAll)+len(y_testAll))
        
    return x_disease_trainAll, x_admin_trainAll,x_dissituation_trainAll,x_treat_trainAll,x_disorder_trainAll,\
        x_disease_testAll,x_admin_testAll,x_dissituation_testAll, x_treat_testAll,x_disorder_testAll,y_trainAll, y_testAll
rate = 0.2
admin = "admission_situationKey"
dissituation = "discharge_situationKey"
treat = "treat_processKey"
disorder = "discharge_orderKey"
x_disease_trainAll, x_admin_trainAll,x_dissituation_trainAll,x_treat_trainAll,x_disorder_trainAll,\
    x_disease_testAll,x_admin_testAll,x_dissituation_testAll, x_treat_testAll,x_disorder_testAll,y_trainAll, y_testAll= classifyDatasetsTFAll( \
    pd_dia_dis,admin,dissituation,treat,disorder, rate\
    )



train size= 46242 test size= 10174
sum= 56416


In [62]:
print("x_disease_train:",len(x_disease_trainAll))
print("x_admin_train:",len(x_admin_trainAll))
print("x_dissituation_train:", len(x_dissituation_trainAll))
print("x_treat_train:", len(x_treat_trainAll))
print("x_disorder_train:", len(x_disorder_trainAll))

x_disease_train: 46242
x_admin_train: 46242
x_dissituation_train: 46242
x_treat_train: 46242
x_disorder_train: 46242


In [68]:
def keywordpad(inStr,maxlen,dtype='int32',value=0.):
    inStrArr = str(inStr).strip().split(" ")
    trunc = inStrArr[:maxlen]
    x = ["0" for i in range(maxlen)]
    x[:len(trunc)] = trunc
    return " ".join(x)

### 构造CNN四个字段的数据集合

In [69]:
def doPrepareCNNAll(x_disease_trainAll, x_admin_trainAll,x_dissituation_trainAll,x_treat_trainAll,x_disorder_trainAll, y_trainAll, filePath):
    f = open(filePath+".feature", "w")
    f_label =open(filePath+".label" ,"w")
    for i in range (len(x_disease_trainAll)):
        dis = x_disease_trainAll[i]
        adm = x_admin_trainAll[i]
        dissituation = x_dissituation_trainAll[i]
        treat = x_treat_trainAll[i]
        disorder = x_disorder_trainAll[i]
        y_train = y_trainAll[i]
        
        y_trainStr = ("%s " %(y_train))
        #featureStr = ("%s%s" %(str(dis)+" " + str(adm) + " __label__", y_trainStr))
                               
        featureStr = ("%s" %(str(dis)+" " + str( keywordpad(adm,20))+" "+str(keywordpad(dissituation,20))+" "+str(keywordpad(treat,20))+" "+str(keywordpad(disorder,20)) ))

        labelStr = ("%s" %(y_trainStr))
        f.write(str(featureStr)+"\n")
        f_label.write(labelStr+"\n")
    f.flush()
    f.close()

filePath = "cnnModel/datasets/trainall"
doPrepareCNNAll(x_disease_trainAll, x_admin_trainAll,x_dissituation_trainAll,x_treat_trainAll,x_disorder_trainAll, y_trainAll, filePath )
filePathtest = "cnnModel/datasets/testall"
doPrepareCNNAll(x_disease_testAll,x_admin_testAll,x_dissituation_testAll,x_treat_testAll,x_disorder_testAll,y_testAll, filePathtest )